# Thunderbird Demo

In [1]:
from birdy import WPSClient
import requests

In [2]:
# Set up wps application
url = 'http://localhost:5001/wps'
thunderbird = WPSClient(url=url)

In [3]:
# Check wps info
thunderbird?

Type:            WPSClient
String form:     <birdy.client.base.WPSClient object at 0x7f04f81b6160>
File:            ~/thunderbird-venv/lib/python3.6/site-packages/birdy/client/base.py
Docstring:      
A Web Processing Service for Climate Explorer data preparation

Processes
---------

generate_climos
    Generate files containing climatological means from input files of daily, monthly, or yearly data that adhere to the PCIC metadata standard (and consequently to CMIP5 and CF standards).

generate_prsn
    Generate precipitation as snow file from precipitation and minimum/maximum temperature data

update_metadata
    Update file containing missing, invalid, or incorrectly named global or variable metadata attributes

split_merged_climos
    Split climo means files into one file per time interval

hello
    Just says a friendly Hello.Returns a literal string output with Hello plus the inputed name.
Class docstring:
Returns a class where every public method is a WPS process available at
t

In [4]:
# Check info on `generate_climos` process
thunderbird.generate_climos?

Signature:
thunderbird.generate_climos(
    netcdf,
    operation,
    dry_run=None,
    convert_longitudes=True,
    split_vars=True,
    split_intervals=True,
    climo=None,
    resolutions=None,
)
Docstring:
Generate files containing climatological means from input files of daily, monthly, or yearly data that adhere to the PCIC metadata standard (and consequently to CMIP5 and CF standards).

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
operation : {'mean', 'std'}string
    Operation to perform on the datasets
climo : {'6190', '7100', '8110', '2020', '2050', '2080'}string
    Year ranges
resolutions : {'all', 'yearly', 'seasonal', 'monthly'}string
    Temporal Resolutions
convert_longitudes : boolean
    Transform longitude range from [0, 360) to [-180, 180)
split_vars : boolean
    Generate a separate file for each dependent variable in the file
split_intervals : boolean
    Generate a separate file f

In [5]:
# Dry run process
netcdf = 'http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100.nc'
operation = 'mean'
climo = '6190'
resolutions = 'yearly'
dry_run = True

dry_output = thunderbird.generate_climos(
    netcdf=netcdf, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=dry_run
)

In [6]:
# Process dry run output
req = requests.get(dry_output.get()[0])
print(req.content.decode('utf-8'))

Dry Run
File: http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100.nc
climo_periods: {'6190'}
project: CMIP5
institution: PCIC
model: CanESM2
emissions: historical, rcp85
run: r1i1p1
dependent_varnames: ['fdd']
time_resolution: seasonal
is_multi_year_mean: False



In [7]:
# generate climos
output = thunderbird.generate_climos(
    netcdf=netcdf, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=False
)

In [8]:
from bs4 import BeautifulSoup
req = requests.get(output.get()[0])
process = BeautifulSoup(req.content.decode('utf-8'))
BeautifulSoup(process.prettify()).metalink.file.metaurl

<metaurl mediatype="application/x-netcdf">
     http://localhost:5001/outputs/676bfd2a-b725-11ea-afd0-c86000e3f2fd/fdd_aClimMean_BCCAQ_CanESM2_historical+rcp85_r1i1p1_19610101-19901231_Canada.nc
    </metaurl>